# Federated learning: Simulation of data poisoning attack

In this notebook provide a simulation of a simple data poisoning federated attack. We use a simple first approach
which consists on shuffling the training labels of some number of clients which become adversarial ones.

The aim of this notebook is to present the class FederatedDataAttack implemented in [link]() whose goal is to 
implement any attack on the federated data. For more information about basic federated learning concepts refers to [link]().

For the simulation we use [Emnist](https://www.nist.gov/itl/products-and-services/emnist-dataset) Digits dataset.

In [3]:
import shfl

database = shfl.data_base.Emnist()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

ModuleNotFoundError: No module named 'emnist'

After that, we distribute the data among the client nodes using a non-iid distribution over 10\% of the data. 

In [ ]:
noniid_distribution = shfl.data_distribution.NonIidDataDistributionIidDataDistribution(database)
federated_data, test_data, test_labels = noniid_distribution.get_federated_data(num_nodes=20, percent=10)

At this point, we are ready to apply some data attack to some nodes. For this simulation, we choose to apply data 
poisoning to the 20\% of the nodes.

In [ ]:
from shfl.private.federated_attack import FederatedPoisoningDataAttack

poisoning_attack = FederatedPoisoningDataAttack(percentage=20)
poisoning_attack.attack(federated_data=federated_data)